<a href="https://colab.research.google.com/github/mkri/master/blob/master/agri_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building FastAi Model

In [1]:
#Printing thw working directory
import os
wd = os.getcwd()

In [2]:
wd

'/content'

In [3]:
# Setting The Home Directory for the project
home_dir = wd[:-7]

In [4]:
home_dir

'/'

In [34]:
#Listing the home directory and working directory contents
print('HOME DIRECTORY')
print(os.listdir(home_dir))
print('WORKING DIRECTORY')
print(os.listdir(wd))

HOME DIRECTORY
['media', 'proc', 'sbin', 'lib64', 'lib', 'tmp', 'var', 'srv', 'bin', 'opt', 'run', 'mnt', 'root', 'home', 'etc', 'boot', 'dev', 'usr', 'sys', 'content', '.dockerenv', 'datalab', 'tools', 'swift', 'tensorflow-1.15.2', 'lib32']
WORKING DIRECTORY
['.config', 'datafileYield.csv', 'sample_data']


## Loading The Datasets

In [70]:
import pandas as pd
import numpy as np
training_set = pd.read_csv(wd+'/datafileYield.csv')

In [71]:
training_set.head(5)

,Crop,Production06-07,Production07-08,Production08-09,Production09-10,Production10-11,Area06-07,Area07-08,Area08-09,Area09-10,Area10-11,Yield06-07,Yield07-08,Yield08-09,Yield09-10,Yield10-11,Location,Price
0,Total Foodgrains,158.8,168.6,171.3,159.4,178.9,128.5,128.8,127.6,126.0,131.7,123.6,130.9,134.3,126.5,135.9,Mumbai,1.75
1,Rice,200.8,207.9,213.3,191.6,206.4,168.5,168.9,175.1,161.2,164.8,119.2,123.1,121.8,118.9,125.2,Pune,12.50
2,Wheat,131.6,136.4,140.1,140.3,150.8,115.0,115.2,114.0,116.9,119.5,114.4,118.4,122.8,120.0,126.3,Chennai,4.50
3,Jowar,124.3,137.8,126.0,116.5,121.8,120.7,110.6,107.3,111.0,105.2,103.0,124.6,117.4,105.0,115.8,Chennai,6.00
4,Bajra,136.4,161.5,143.9,105.4,167.9,94.5,95.1,87.0,88.5,95.6,144.3,169.7,165.4,119.0,175.8,Coimbatore,17.74


## Structuring & Formatting The Datasets

In [ ]:
def restructure(data):
  
  names = list(data.Name)
  
  brand = []
  model = []
  
  for i in range(len(names)):
    try:
      brand.append(names[i].split(" ")[0])
      try:
        model.append(" ".join(names[i].split(" ")[1:]).strip())
      except:
        pass
    except:
        print("ERR ! - ", names[i], "@" , i)
        
        
  mileage = list(data.Mileage)
  
  for i in range(len(mileage)):
    try :
      mileage[i] = float(mileage[i].split(" ")[0].strip())
    except:
      mileage[i] = np.nan
      
      
  engine = list(data.Engine)
  for i in range(len(engine)):
    try :
      engine[i] = int(engine[i].split(" ")[0].strip())
    except:
      engine[i] = np.nan
      
      
  power = list(data.Power)
  for i in range(len(power)):
    try :
      power[i] = float(power[i].split(" ")[0].strip())
    except:
      power[i] = np.nan
      
  data['New_Price'].fillna(0, inplace = True)
  
  newp = list(data['New_Price'])
  
  for i in range(len(newp)):
    if newp[i] == 0:
      newp[i] = float(newp[i])
      continue
    elif 'Cr' in newp[i]:
      newp[i] = float(newp[i].split()[0].strip()) * 100 
    elif 'Lakh' in newp[i]:
      newp[i] = float(newp[i].split()[0].strip())
      
      
#Re-ordering the columns

  restructured = pd.DataFrame({'Brand': brand,
                              'Model':model,
                              'Location': data['Location'], 
                              'Year':data['Year'] , 
                              'Kilometers_Driven':data['Kilometers_Driven'],
                              'Fuel_Type':data['Fuel_Type'],
                              'Transmission':data['Transmission'],
                              'Owner_Type':data['Owner_Type'],
                              'Mileage':mileage,
                              'Engine':engine,
                              'Power':power,
                              'Seats':data['Seats'],
                              'New_Price':newp
                             })

  if 'Price' in data.columns:
    restructured['Price'] = data['Price']
    return restructured

  else:
    return restructured

In [ ]:
train_d = restructure(training_set)


### Selecting Few Features

In [ ]:
cols = ['Brand', 'Location', 'Year', 'Kilometers_Driven', 'Fuel_Type',
       'Transmission', 'Owner_Type', 'Mileage', 'Price']

In [72]:
cols = ['Crop', 'Production06-07', 'Production07-08', 'Production08-09', 'Production09-10','Production10-11',
       'Area06-07', 'Area07-08', 'Area08-09', 'Area09-10', 'Area10-11','Yield06-07', 'Yield07-08', 'Yield08-09', 'Yield09-10', 'Yield10-11', 'Location', 'Price']

In [73]:
train_d = training_set[cols]

In [74]:
train_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Crop             98 non-null     object 
 1   Production06-07  98 non-null     float64
 2   Production07-08  98 non-null     float64
 3   Production08-09  98 non-null     float64
 4   Production09-10  98 non-null     float64
 5   Production10-11  98 non-null     float64
 6   Area06-07        98 non-null     float64
 7   Area07-08        98 non-null     float64
 8   Area08-09        98 non-null     float64
 9   Area09-10        98 non-null     float64
 10  Area10-11        98 non-null     float64
 11  Yield06-07       98 non-null     float64
 12  Yield07-08       98 non-null     float64
 13  Yield08-09       98 non-null     float64
 14  Yield09-10       98 non-null     float64
 15  Yield10-11       98 non-null     float64
 16  Location         98 non-null     object 
 17  Price            9

## Modeling With Fast.ai


In [75]:
from fastai.tabular import *
#This path will be used for saving and exporting the model
path = wd

In [76]:
#The target variable that we are trying to predict
dep_var = 'Price'

#The categorical variables 
cat_names = list(train_d.select_dtypes('object').columns)

#The continuous variables
cont_names =['Production06-07', 'Area10-11', 'Yield10-11'] #No need to keep the Dependend variable

#Preprocessing steps for the fastai learner
procs = [FillMissing, Categorify, Normalize]

In [77]:
#Creating a validation set
val = TabularList.from_df(train_d.iloc[800:1000].copy(), path=path, cat_names=cat_names, cont_names=cont_names)

In [78]:
#Creating a trainig set
data = (TabularList.from_df(train_d, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(len(train_d) - int(len(train_d) * 0.2),len(train_d))))
                           .label_from_df(cols=dep_var)
                           .add_test(val)
                           .databunch())

In [79]:
data.show_batch(5)

Crop,Location,Production06-07,Area10-11,Yield10-11,target
Ragi,Jaipur,-0.5900,-1.1759,-0.2348,5.2
Maize,Hyderabad,0.0964,0.7591,-0.0238,17.5
Dry chilies,Delhi,-0.0734,-0.7825,0.0638,7.5
Garlic,Kolkata,0.2282,2.0273,-0.2321,8.5
Total Fruits & Vegetables,Jaipur,-0.0777,1.1188,-0.1488,2.95


### Initializing Neural Network

In [80]:
learn = tabular_learner(data, layers=[300,100, 100, 50], metrics= rmse)

### Training The Model

In [81]:
learn.fit(25, 1e-2)

epoch,train_loss,valid_loss,root_mean_squared_error,time
0,278.857666,313.619720,17.709312,00:00
1,293.999573,313.028534,17.692612,00:00
2,270.017700,313.592621,17.708546,00:00
3,258.186035,316.046631,17.777700,00:00
4,266.252289,320.980072,17.915916,00:00
5,268.839661,327.810944,18.105551,00:00
6,255.260284,337.067383,18.359394,00:00
7,253.389801,349.895447,18.705492,00:00
8,244.962082,364.436890,19.090231,00:00
9,242.061676,380.792572,19.513906,00:00


In [82]:
learn.show_results(ds_type=DatasetType.Train)

Crop,Location,Production06-07,Area10-11,Yield10-11,target,prediction
Coffee,Pune,-0.1961,-0.0594,-0.2407,1.25,[-14.052549]
Rice,Pune,0.1060,0.6786,-0.2583,6.0,[-10.750392]
Coarse Cereals,Chennai,-0.1937,-0.4341,-0.0233,4.49,[-12.942563]
Onion,Delhi,0.2970,3.4865,-0.2327,10.31,[-1.127859]
Linseed,Kochi,-0.5910,-1.6704,-0.2652,18.55,[-0.999559]


In [83]:
learn.show_results(ds_type=DatasetType.Valid)

Crop,Location,Production06-07,Area10-11,Yield10-11,target,prediction
Cereals,Kochi,-0.0705,0.0230,-0.1734,5.6,[-10.600332]
Gram,Kolkata,-0.1836,0.3995,-0.2770,27.0,[-13.243777]
Arhar,Jaipur,-0.3933,-0.1194,-0.4031,17.5,[-11.585635]
Other Pulses,Delhi,-0.1908,0.1972,-0.2380,15.0,[-2.476892]
Total Pulses,Pune,-0.2331,0.1953,-0.2839,5.4,[-7.254779]


## Saving & Exporting The Model

In [84]:
learn.save('model',return_path=True)

PosixPath('/content/models/model.pth')

In [85]:
learn.export('model.pkl')

In [86]:
model = load_learner(path, 'model.pkl')